In [ ]:
from openfermion.hamiltonians import fermi_hubbard
from openfermion.ops import QubitOperator, FermionOperator
from openfermion.utils import chemist_ordered
from openfermion.transforms import get_sparse_operator
#from openfermion.linalg import eigenspectrum
import sys
from os import cpu_count
from time import time
sys.path.append("../")
from mappings import hubbard_mapping_helper as mapper
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import animation
import seaborn as sns
import numpy as np
from scipy.optimize import minimize

from math import pi
from copy import deepcopy
from simulator.logical_vector_simulator import logical_state_simulation#simulate_parallel, prepare_stabilizers
from simulator.logical_vector_construction import reduce_to_logical_plus
from simulator.noise import single_qubit_depolarizing, parallel_single_qubit_depolarizing, noiseless
from simulator.utils import expectation_value, check_mapping, anticom, fidelity
import pickle
from random import shuffle
from itertools import combinations, product

#### Non-equilibrium dynamics experiments

In [ ]:
mappings = ['jw','lw_15','gse_20','aqm'] 
mapping_shots = {
    'jw':10000,
    'lw_15':10000,
    'gse_20':10000,
    'aqm':10000
}
num_layers = 16

t = (5/2)*pi 
delta = 0
init_occupied_modes = [5,6]

shots = 10000
stat_noise_rate = 0
gate_noise_rate = 0.01
if stat_noise_rate == 0.0:
    stat_noise_model = noiseless()
else:
    stat_noise_model = single_qubit_depolarizing(stat_noise_rate)
if gate_noise_rate == 0.0:
    gate_noise_model = noiseless()
else:
    gate_noise_model = single_qubit_depolarizing(gate_noise_rate)
    gate_noise_model = parallel_single_qubit_depolarizing(gate_noise_rate)

cpus = cpu_count()-2

In [ ]:
mapping_hams = {}
mapping_stabs = {}
mapping_Aops = {}
mapping_Bops = {}
mapping_qubits = {
    'jw':12,
    'aqm':25,
    'gse_20':20,
    'lw_15':15}
mapping_fixed_ops = {}
mapping_circuit = {
    'jw':mapper.jw_circuit,
    'aqm':mapper.aqm_circuit,
    'gse_20':mapper.gse_20_circuit,
    'lw_15':mapper.lw_15_circuit}
mapping_mappers = {
    'jw':mapper.jw_mapping,
    'aqm':mapper.aqm_mapping,
    'gse_20':mapper.gse_20_mapping,
    'lw_15':mapper.lw_15_mapping}
mapping_syndromes = {
    'jw':[],
    'aqm':[],
    'gse_20':[],
    'lw_15':[]}

for mapping in mappings:
    mapping_hams[mapping], mapping_stabs[mapping], mapping_Aops[mapping], mapping_Bops[mapping] = mapping_mappers[mapping](FermionOperator())

mapping_stabs['jw'] = []
# fix the vacuum for all mappings
for mapping in mappings:
    mapping_fixed_ops[mapping] = []
    for j in range(12):
        if j in init_occupied_modes:
            mapping_fixed_ops[mapping].append((-1)*mapping_Bops[mapping][j])
        else:
            mapping_fixed_ops[mapping].append(mapping_Bops[mapping][j])
    # Remove the last one for the mappings where global parity is included in stabilizers
    if mapping_qubits[mapping] - len(mapping_stabs[mapping]) < 12:
        mapping_fixed_ops[mapping] = mapping_fixed_ops[mapping][:-1]

for mapping in mappings:
    print(mapping)
    check_mapping(mapping_Aops[mapping],mapping_Bops[mapping], mapping_stabs[mapping])

In [ ]:
avg_Aop_weights = {}
avg_Bop_weights = {}
avg_stab_weights = {}

def op_weight(op):
    return len(list(op.terms.items())[0][0])

for mapping in mappings:
    Aop_list = list(mapping_Aops[mapping].items())
    summed_weight = 0
    for j in range(len(Aop_list)):
        summed_weight += op_weight(Aop_list[j][1])
    avg_Aop_weights[mapping] = summed_weight/len(Aop_list)

    Bop_list = list(mapping_Bops[mapping].items())
    summed_weight = 0
    for j in range(len(Bop_list)):
        summed_weight += op_weight(Bop_list[j][1])
    avg_Bop_weights[mapping] = summed_weight/len(Bop_list)

    summed_weight = 0
    for j in range(len(mapping_stabs[mapping])):
        summed_weight += op_weight(mapping_stabs[mapping][j])
    if len(mapping_stabs[mapping])>0:
        avg_stab_weights[mapping] = summed_weight/len(mapping_stabs[mapping])
    else:
        avg_stab_weights[mapping] = 0

In [ ]:
print(avg_Aop_weights)
print(avg_Bop_weights)
print(avg_stab_weights)

In [ ]:
if delta == 0:
    circ_layers = [0 for j in range(4)]
else:
    circ_layers = [0 for j in range(5)]

circ_layers[0] = [FermionOperator('1^ 6', pi/2) + FermionOperator('6^ 1',pi/2)]
circ_layers[0].append(FermionOperator('2^ 3', pi/2) + FermionOperator('3^ 2',pi/2))
circ_layers[0].append(FermionOperator('8^ 9', pi/2) + FermionOperator('9^ 8',pi/2))
circ_layers[0].append(FermionOperator('5^ 10', pi/2) + FermionOperator('10^ 5',pi/2))

circ_layers[1] = [FermionOperator('0^ 2', t) + FermionOperator('2^ 0',t)]
circ_layers[1].append(FermionOperator('6^ 8', t) + FermionOperator('8^ 6',t))
circ_layers[1].append(FermionOperator('3^ 5', t) + FermionOperator('5^ 3',t))
circ_layers[1].append(FermionOperator('9^ 11', t) + FermionOperator('11^ 9',t))

circ_layers[2] = [FermionOperator('0^ 1', t) + FermionOperator('1^ 0',t)]
circ_layers[2].append(FermionOperator('6^ 7', t) + FermionOperator('7^ 6',t))
circ_layers[2].append(FermionOperator('3^ 8', t) + FermionOperator('8^ 3',t))
circ_layers[2].append(FermionOperator('4^ 5', t) + FermionOperator('5^ 4',t))
circ_layers[2].append(FermionOperator('10^ 11', t) + FermionOperator('11^ 10',t))

circ_layers[3] = [FermionOperator('1^ 3', t) + FermionOperator('3^ 1',t)]
circ_layers[3].append(FermionOperator('7^ 9', t) + FermionOperator('9^ 7',t))
circ_layers[3].append(FermionOperator('2^ 4', t) + FermionOperator('4^ 2',t))
circ_layers[3].append(FermionOperator('8^ 10', t) + FermionOperator('10^ 8',t))

if delta !=0:
    # A sublattice sites: 0,6,3,9,4,10  -delta
    circ_layers[4] = []
    for j in [0,6,3,9,4,10]:
        circ_layers[4].append(FermionOperator(f'{j}^ {j}', -1*delta))
    for j in [1,7,2,8,5,11]:
        circ_layers[4].append(FermionOperator(f'{j}^ {j}', delta))

In [ ]:
ferm_observables = [FermionOperator(f'{j}^ {j}') for j in range(12)]
mapping_observables = {}
for mapping in mappings:
    mapping_observables[mapping] = {}
for j in range(12):
    for mapping in mappings:
        temp_obs, _, _, _ = mapping_mappers[mapping](FermionOperator(f'{j}^ {j}'),up_down_reorder=False)
        mapping_observables[mapping][j] = temp_obs

#### function for running experiment

In [ ]:
def dynamics_simulation(mapping, circ_layers):

    num_expects = np.zeros((12,num_layers+1))
    fidelities = [0 for j in range(num_layers+1)]
    traces = [0 for j in range(num_layers+1)]
    depths = [0 for j in range(num_layers+1)]

    zero_block_entries_count = 0
    for j in range(num_layers + 1):
        print(f'{mapping} simulation: layer {j+1}/{num_layers+1}, zero_block_entries_count = {zero_block_entries_count}')
        ferm_circuit = []
        for k in range(j):
            ferm_circuit = ferm_circuit + circ_layers[k % 4]
        schedule = mapping_circuit[mapping](ferm_circuit)
        block_entries = 0
        attempts = 0
        while block_entries == 0:
            attempts +=1
            (state, block_entries, ideal_state, depth, data)  = logical_state_simulation(
                stabilizers = mapping_stabs[mapping],
                n_phys_qubits = mapping_qubits[mapping],
                rounds = mapping_shots[mapping],
                stat_noise = stat_noise_model,
                gate_noise = gate_noise_model,
                phys_circuit = schedule,
                logical_operators = mapping_fixed_ops[mapping],
                d_matrix_blocks = 'code',
                block_numbers = [],
                num_processes = cpus
                )
            if block_entries == 0:
                zero_block_entries_count += 1
        updated_stabs = data[0]
        qubit_order = data[1]
        fixed_positions = data[2]
        fixed_ops = data[3]
        other_ops = data[4]

        codespace_state = state
        codespace_trace = np.real(np.trace(codespace_state))
        codespace_state = (1/codespace_trace)*codespace_state
        fidelities[j] = fidelity(codespace_state,ideal_state)

        traces[j] = np.real(np.trace(codespace_state)) * block_entries/(attempts*mapping_shots[mapping])
        depths[j] = depth[0] +  depth[1]

        for k in range(12):
            num_expects[k,j] = np.real(expectation_value(
            mapping_observables[mapping][k],
            state,
            updated_stabs,
            qubit_order,
            fixed_positions, 
            fixed_ops, 
            other_ops
            ))
    
    return num_expects, fidelities, traces, depths

#### Data collection

In [ ]:
num_expects = {}
fidelities = {}
traces = {}
depths = {}
for mapping in mappings :
    num_expects[mapping], fidelities[mapping], traces[mapping], depths[mapping] = dynamics_simulation(mapping, circ_layers)

In [ ]:
summed_num_expects = {}
for mapping in mappings:
    summed_num_expects[mapping] = [0 for j in range(num_expects[mapping].shape[1])]
    for timestep in range(num_expects[mapping].shape[1]):
        summed_num_expects[mapping][timestep] = np.sum(num_expects[mapping][:,timestep])

In [ ]:
number_maps= {}
for mapping in mappings:
    number_maps[mapping] = np.zeros((3,4,num_layers+1))
for mapping in mappings:
    for j in range(num_layers+1):
        number_maps[mapping][0,0,j] = num_expects[mapping][0,j]
        number_maps[mapping][0,1,j] = num_expects[mapping][1,j]
        number_maps[mapping][0,2,j] = num_expects[mapping][6,j]
        number_maps[mapping][0,3,j] = num_expects[mapping][7,j]
        number_maps[mapping][1,0,j] = num_expects[mapping][2,j]
        number_maps[mapping][1,1,j] = num_expects[mapping][3,j]
        number_maps[mapping][1,2,j] = num_expects[mapping][8,j]
        number_maps[mapping][1,3,j] = num_expects[mapping][9,j]
        number_maps[mapping][2,0,j] = num_expects[mapping][4,j]
        number_maps[mapping][2,1,j] = num_expects[mapping][5,j]
        number_maps[mapping][2,2,j] = num_expects[mapping][10,j]
        number_maps[mapping][2,3,j] = num_expects[mapping][11,j]


#### Animation of chiral mode propagation

In [ ]:
for mapping in mappings:
    fig, ax = plt.subplots(1,1,figsize=(8,6),dpi=200)
    img = ax.imshow(number_maps[mapping][:,:,0],cmap='magma')
    def init():
        img = ax.imshow(number_maps[mapping][:,:,j],cmap='magma')
        plt.title(f'Floquet edge modes, timestep 0/{num_layers}')
        ax.set_xticks([0,1,2,3])
        ax.set_yticks([0,1,2])
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        
    def animate(j):
        img = ax.imshow(number_maps[mapping][:,:,j],cmap='magma')
        plt.title(f'Floquet edge modes, timestep {j}/{num_layers}')
        ax.set_xticks([0,1,2,3])
        ax.set_yticks([0,1,2])
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        
    cbar = fig.colorbar(img)
    cbar.ax.set_ylabel(r'$\langle \hat n_{x,y} \rangle$', fontsize=12, rotation=0)
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=num_layers+1, repeat=False)

    anim.save(f'{mapping}_animated_QCA_noisy_parallel_noise.png')
    plt.close()

#### Plotting

In [ ]:
labels ={
    'jw': r'JW1 $[[12,12,1]]$',
    'aqm': r'GAQM $[[25,12,2]]$',
    'gse_20': r'GSE $[[20,12,1]]$',
    'lw_15': r'Compact $[[15,12,1]]$'
}
plt.style.use('ggplot')
colors = {}
for j in range(len(mappings)):
    colors[mappings[j]] = list(plt.rcParams['axes.prop_cycle'])[j]['color']

fig = plt.figure(figsize=(8,6),dpi=200)


for mapping in mappings:
    plt.scatter(range(num_layers+1),fidelities[mapping],label = labels[mapping],marker = '+',color=colors[mapping])

#plt.scatter(range(num_layers+1),fidelities['jw'],label = 'JW (12 qubits)',marker = '+',color=list(plt.rcParams['axes.prop_cycle'])[0]['color'])
#plt.scatter(range(num_layers+1),fidelities['aqm'],label = 'AQM (25 qubits)',marker = '+',color=list(plt.rcParams['axes.prop_cycle'])[3]['color'])
#plt.scatter(range(num_layers+1),fidelities['gse_20'],label = 'GSE (20 qubits)',marker = '+',color=list(plt.rcParams['axes.prop_cycle'])[5]['color'])

plt.ylabel('fidelity')
plt.xlabel('timesteps')
plt.legend(loc='best')
plt.show()


In [ ]:
fig = plt.figure(figsize=(8,6),dpi=200)


for mapping in mappings:
    plt.scatter(range(num_layers+1),depths[mapping],label = labels[mapping],marker = '+',color=colors[mapping])

#plt.scatter(range(num_layers+1),fidelities['jw'],label = 'JW (12 qubits)',marker = '+',color=list(plt.rcParams['axes.prop_cycle'])[0]['color'])
#plt.scatter(range(num_layers+1),fidelities['aqm'],label = 'AQM (25 qubits)',marker = '+',color=list(plt.rcParams['axes.prop_cycle'])[3]['color'])
#plt.scatter(range(num_layers+1),fidelities['gse_20'],label = 'GSE (20 qubits)',marker = '+',color=list(plt.rcParams['axes.prop_cycle'])[5]['color'])

plt.ylabel('preparation and circuit depth')
plt.xlabel('timesteps')
plt.legend(loc='best')
plt.show()

In [ ]:
sampling_increase = {}

for j in range(len(mappings)):
    sampling_increase[mappings[j]] = [0 for j in range(len(traces[mappings[j]]))]
    for k in range(len(traces[mappings[j]])):
        #sampling_increase[j,k] = 1/(averaged_traces[j,k]*averaged_fidelities[j,k]**2)
        sampling_increase[mappings[j]][k] = 1/traces[mappings[j]][k]

unmitigated_fidelities = {}
for mapping in mappings:
    unmitigated_fidelities[mapping] = [0 for j in range(len(traces[mapping]))]
    for j in range(len(traces[mapping])):
        unmitigated_fidelities[mapping][j] = traces[mapping][j] * fidelities[mapping][j]

In [ ]:
import matplotlib as mpl
mpl.rc_file_defaults()
plt.style.use('ggplot')
#plt.rc('font', family='serif')
#plt.rc('text', usetex=True)
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
#fig = plt.figure(figsize=(6,4),dpi=200)#figsize=(8,6),dpi=200)#
#ax = fig.add_subplot(111)
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True,
                                    figsize=(12, 6),dpi=300)
fig.subplots_adjust(wspace = 0.22)
for j in range(len(mappings)):
    ax0.plot(range(num_layers+1),depths[mappings[j]],label = labels[mappings[j]],marker='+',color=colors[mappings[j]]) #
ax0.set_xlabel(f'timesteps')
ax0.set_ylabel(f'depth')
ax0.xaxis.label.set_color('black')
ax0.yaxis.label.set_color('black')
ax0.tick_params(axis='x', colors='black')
ax0.tick_params(axis='y', colors='black')
#ax0.margins(x=0)

for j in range(len(mappings)):
    ax1.plot(range(num_layers+1),fidelities[mappings[j]],label = labels[mappings[j]],marker='+',color=colors[mappings[j]])
    ax1.plot(range(num_layers+1),unmitigated_fidelities[mappings[j]],linestyle='dashed',marker='+',color=colors[mappings[j]])
ax1.set_xlabel(f'timesteps')
ax1.set_ylabel(f'codespace fidelity')
#ax1.set_yscale('log')
ax1.xaxis.label.set_color('black')
ax1.yaxis.label.set_color('black')
ax1.tick_params(axis='x', colors='black')
ax1.tick_params(axis='y', colors='black')
dummy1 = ax1.plot([],color=colors[mappings[2]],label='mitigated',marker='+')
dummy2 = ax1.plot([],color=colors[mappings[2]],linestyle='dashed',label='unmitigated',marker='+')
ax1.legend(labels=['mitigated','unmitigated'])

for j in range(len(mappings)):
    ax2.plot(range(num_layers+1),traces[mappings[j]],label = labels[mappings[j]],marker='+',color=colors[mappings[j]])
ax2.set_xlabel(f'timesteps')
ax2.set_ylabel(f'codespace trace')
ax2.set_yscale('log')
ax2.xaxis.label.set_color('black')
ax2.yaxis.label.set_color('black')
ax2.tick_params(axis='x', colors='black')
ax2.tick_params(axis='y', colors='black')

legend = plt.legend(loc='upper center', bbox_to_anchor=(-.78,1.081), ncol=len(mappings), fancybox=True, shadow=False)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('white')
#plt.show()
plt.savefig('chiral_mode_figure.png',facecolor='white')

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True,
                                    figsize=(12, 4),dpi=1000)
fig.subplots_adjust(wspace = 0.22)
for j in range(len(mappings)):
    ax0.plot(range(num_layers+1),depths[mappings[j]],label = labels[mappings[j]],marker='+',color=colors[mappings[j]]) #
ax0.set_xlabel(f'timesteps')
ax0.set_ylabel(f'depth')
ax0.xaxis.label.set_color('black')
ax0.yaxis.label.set_color('black')
ax0.tick_params(axis='x', colors='black')
ax0.tick_params(axis='y', colors='black')
#ax0.margins(x=0)

ax1.plot(range(num_layers+1),fidelities[mappings[0]],linestyle='dashed',label = labels[mappings[0]],marker='+',color=colors[mappings[0]])
for j in range(1,len(mappings)):
    ax1.plot(range(num_layers+1),fidelities[mappings[j]],label = labels[mappings[j]],marker='+',color=colors[mappings[j]])
    ax1.plot(range(num_layers+1),unmitigated_fidelities[mappings[j]],linestyle='dashed',marker='+',color=colors[mappings[j]])
ax1.set_xlabel(f'timesteps')
ax1.set_ylabel(f'codespace fidelity')
#ax1.set_yscale('log')
ax1.xaxis.label.set_color('black')
ax1.yaxis.label.set_color('black')
ax1.tick_params(axis='x', colors='black')
ax1.tick_params(axis='y', colors='black')
dummy1, = ax1.plot([],color='black',label='mitigated',marker='+')
dummy2, = ax1.plot([],color='black',linestyle='dashed',label='unmitigated',marker='+')
#ax1.legend(labels=['mitigated','unmitigated'])
dummy1.set_color('gray')
dummy2.set_color('gray')
ax1.legend(handles=[dummy1,dummy2])

for j in range(len(mappings)):
    ax2.plot(range(num_layers+1),sampling_increase[mappings[j]],label = labels[mappings[j]],marker='+',color=colors[mappings[j]])
ax2.set_xlabel(f'timesteps')
ax2.set_ylabel(f'sampling increase factor')
ax2.set_yscale('log')
ax2.xaxis.label.set_color('black')
ax2.yaxis.label.set_color('black')
ax2.tick_params(axis='x', colors='black')
ax2.tick_params(axis='y', colors='black')

legend = plt.legend(loc='upper center', bbox_to_anchor=(-.78,1.11), ncol=len(mappings), fancybox=True, shadow=False)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('white')
#plt.show()
plt.savefig('chiral_mode_figure.png',facecolor='white')

In [ ]:
perimeter_expects = {}
for mapping in mappings:
    perimeter_expects[mapping] = np.zeros((10,num_layers+1))
    perimeter_expects[mapping][0,:] = number_maps[mapping][0,0,:]
    perimeter_expects[mapping][1,:] = number_maps[mapping][0,1,:]
    perimeter_expects[mapping][2,:] = number_maps[mapping][0,2,:]
    perimeter_expects[mapping][3,:] = number_maps[mapping][0,3,:]
    perimeter_expects[mapping][4,:] = number_maps[mapping][1,3,:]
    perimeter_expects[mapping][5,:] = number_maps[mapping][2,3,:]
    perimeter_expects[mapping][6,:] = number_maps[mapping][2,2,:]
    perimeter_expects[mapping][7,:] = number_maps[mapping][2,1,:]
    perimeter_expects[mapping][8,:] = number_maps[mapping][2,0,:]
    perimeter_expects[mapping][9,:] = number_maps[mapping][1,0,:]

#### Save data

In [ ]:
data_dict = {
    'mappings':mappings,
    'num_layers': num_layers,
    'num_expects':num_expects, 
    'fidelities':fidelities, 
    'number_maps':number_maps, 
    'perimeter_expects': perimeter_expects
}
with open(f'QCA_data_final.pkl', 'wb') as f:
   pickle.dump(data_dict,f)

In [ ]:
mpl.rc_file_defaults()
plt.style.use('ggplot')
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
fig, ax = plt.subplots(2,2,figsize=(5,6),dpi=600)
ax[0,0].imshow(perimeter_expects['jw'].T,cmap='magma')
#ax[0,0].set_xlabel('perimeter mode')
ax[0,0].set_ylabel('timesteps', color='black')
ax[0,0].invert_yaxis()
ax[0,0].set_xticks([])
ax[0,0].set_yticks([0,2,4,6,8,10,12,14,16])
ax[0,0].set_title(labels['jw'],fontsize=10)
ax[0,0].grid(visible=False)
ax[0,0].tick_params(axis='y', colors='black')


ax[0,1].imshow(perimeter_expects['lw_15'].T,cmap='magma')
#ax[0,1].set_xlabel('perimeter mode')
#ax[0,1].set_ylabel('timesteps')
ax[0,1].invert_yaxis()
ax[0,1].set_xticks([])
ax[0,1].set_yticks([])
ax[0,1].set_title(labels['lw_15'],fontsize=10)
ax[0,1].grid(visible=False)



ax[1,0].imshow(perimeter_expects['gse_20'].T,cmap='magma')
ax[1,0].set_xlabel('perimeter mode', color='black')
ax[1,0].set_ylabel('timesteps', color='black')
ax[1,0].invert_yaxis()
ax[1,0].set_xticks([0,2,4,6,8])
ax[1,0].set_yticks([0,2,4,6,8,10,12,14,16])
ax[1,0].set_title(labels['gse_20'],fontsize=10)
ax[1,0].grid(visible=False)
ax[1,0].tick_params(axis='y', colors='black')
ax[1,0].tick_params(axis='x', colors='black')

im = ax[1,1].imshow(perimeter_expects['aqm'].T,cmap='magma')
ax[1,1].set_xlabel('perimeter mode', color='black')
ax[1,1].invert_yaxis()
ax[1,1].set_xticks([0,2,4,6,8])
ax[1,1].set_yticks([])
ax[1,1].set_title(labels['aqm'],fontsize=10)
ax[1,1].grid(visible=False)
ax[1,1].tick_params(axis='x', colors='black')

cbar_ax = fig.colorbar(im, ax=ax.ravel().tolist()) #,label=r'$\langle \hat n_{x,y} \rangle$'
#fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
cbar_ax.set_label(r'$\langle \hat n \rangle$', fontsize=14, rotation=0,color='black')
#cbar_ax.ax.axes.set_tick_params(color='black')
cbar_yticks = plt.getp(cbar_ax.ax.axes, 'yticklabels')
plt.setp(cbar_yticks, color='black')
#cbar_ax.label(r'$\langle \hat n_{x,y} \rangle$', fontsize=12, rotation=0)
#fig.colorbar(im, cax=cbar_ax)

plt.show()


In [ ]:
mpl.rc_file_defaults()
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
fig, ax = plt.subplots(2,2,dpi=200,figsize=(6,5))
ax[0,0].imshow(perimeter_expects['jw'],cmap='magma')
#ax[0,0].set_xlabel('perimeter mode')
ax[0,0].set_ylabel('perimeter mode')
ax[0,0].invert_yaxis()
#ax[0,0].set_xticks([])
ax[0,0].set_yticks([0,2,4,6,8])
ax[0,0].set_title(labels['jw'])


ax[0,1].imshow(perimeter_expects['lw_15'],cmap='magma')
#ax[0,1].set_xlabel('perimeter mode')
#ax[0,1].set_ylabel('timesteps')
ax[0,1].invert_yaxis()
#ax[0,1].set_xticks([])
ax[0,1].set_yticks([0,2,4,6,8])
ax[0,1].set_title(labels['lw_15'])



ax[1,0].imshow(perimeter_expects['gse_20'],cmap='magma')
ax[1,0].set_ylabel('perimeter mode')
ax[1,0].set_xlabel('timesteps')
ax[1,0].invert_yaxis()
ax[1,0].set_yticks([0,2,4,6,8])
ax[1,0].set_title(labels['gse_20'])

ax[1,1].imshow(perimeter_expects['aqm'],cmap='magma')

ax[1,1].set_xlabel('timesteps')
ax[1,1].invert_yaxis()
ax[1,1].set_yticks([0,2,4,6,8])
ax[1,1].set_title(labels['aqm'])

plt.show()

In [ ]:
plt.style.use('ggplot')
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
fig = plt.figure(figsize=(8,5),dpi=1000)
for mapping in mappings:
    plt.plot(range(num_layers+1),summed_num_expects[mapping], label=labels[mapping],color = colors[mapping],marker='+')
plt.xlabel('timesteps',color='black')
plt.ylabel(r'$\hat{N}_{total}$',color='black')
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')
legend = plt.legend(loc='upper center', bbox_to_anchor=(0.48,1.08), ncol=len(mappings), fancybox=True, shadow=False,fontsize=10)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('white')
#plt.show()
plt.savefig('floquet_total_numbers.png',facecolor='white')

In [ ]:
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(12,4),dpi=1000)


gs0 = gridspec.GridSpec(1, 2, figure=fig,width_ratios=[1,2])

gs00 = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=gs0[0])

ax1 = fig.add_subplot(gs00[0, 0])
ax2 = fig.add_subplot(gs00[0, 1])
ax3 = fig.add_subplot(gs00[1, 0])
ax4 = fig.add_subplot(gs00[1, 1])

ax1.imshow(perimeter_expects['jw'].T,cmap='magma')
#ax1.set_xlabel('perimeter mode')
ax1.set_ylabel('timesteps', color='black')
ax1.invert_yaxis()
ax1.set_xticks([])
ax1.set_yticks([0,2,4,6,8,10,12,14,16])
ax1.set_title(labels['jw'],fontsize = 9)
ax1.grid(visible=False)
ax1.tick_params(axis='y', colors='black')


ax2.imshow(perimeter_expects['lw_15'].T,cmap='magma')
#ax2.set_xlabel('perimeter mode')
#ax2.set_ylabel('timesteps')
ax2.invert_yaxis()
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_title(labels['lw_15'],fontsize = 9)
ax2.grid(visible=False)



ax3.imshow(perimeter_expects['gse_20'].T,cmap='magma')
ax3.set_xlabel('perimeter mode', color='black')
ax3.set_ylabel('timesteps', color='black')
ax3.invert_yaxis()
ax3.set_xticks([0,2,4,6,8])
ax3.set_yticks([0,2,4,6,8,10,12,14,16])
ax3.set_title(labels['gse_20'],fontsize = 9)
ax3.grid(visible=False)
ax3.tick_params(axis='y', colors='black')
ax3.tick_params(axis='x', colors='black')

im = ax4.imshow(perimeter_expects['aqm'].T,cmap='magma')
ax4.set_xlabel('perimeter mode', color='black')
ax4.invert_yaxis()
ax4.set_xticks([0,2,4,6,8])
ax4.set_yticks([])
ax4.set_title(labels['aqm'],fontsize = 9)
ax4.grid(visible=False)
ax4.tick_params(axis='x', colors='black')
axs = [ax1,ax2,ax3,ax4]
cbar_ax = fig.colorbar(im, ax=axs) #,label=r'$\langle \hat n_{x,y} \rangle$'
#fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
cbar_ax.set_label(r'$\langle \hat n \rangle$', fontsize=12, rotation=0,color='black',position=(10,0.5))
#cbar_ax.ax.axes.set_tick_params(color='black')
cbar_yticks = plt.getp(cbar_ax.ax.axes, 'yticklabels')
plt.setp(cbar_yticks, color='black')

# the following syntax does the same as the GridSpecFromSubplotSpec call above:
gs01 = gs0[1].subgridspec(3, 3)

ax5 = fig.add_subplot(gs0[1])
for mapping in mappings:
    ax5.plot(range(num_layers+1),summed_num_expects[mapping], label=labels[mapping],color = colors[mapping],marker='+')
ax5.set_xlabel('timesteps', color='black')
ax5.set_ylabel(r'$\langle\hat{N}_{total} \rangle$',color='black')
ax5.tick_params(axis='x', colors='black')
ax5.tick_params(axis='y', colors='black')
legend = ax5.legend(loc='upper center', bbox_to_anchor=(0.48,1.1), ncol=len(mappings), fancybox=True, shadow=False,fontsize=9)
frame = legend.get_frame()
frame.set_facecolor('white')
frame.set_edgecolor('white')



plt.show()